# Clinical Trial Data Retrieval (Scrape)

This notebook contains Python script I have written in order to more efficiently examine clinical trials and identify trials that report high occurrences of specific adverse events, neuropathy in this example.  The main set of clinical trials I will use are .xml files downloaded from clinicaltrials.gov [Multiple Myeloma](https://clinicaltrials.gov/ct2/results?cond=Multiple+Myeloma&term=&cntry=&state=&city=&dist=&Search=Search&recrs=e&rslt=With) when searching for completed trials with results.  This search resulted in 311 separate trials that were bulk downloaded.



To start, I imported some the os, numpy, and pandas libraries to work with the data, bs4 (BeautifulSoup) to scrape the HTML-based trial information from the .xml files, and sqlite3 to save the results to a database and perform queries.

In [1]:
import os
import numpy as np
import pandas as pd

import bs4 as bs
import sqlite3

from shutil import copyfile

# Parsing Functions

These first four functions were created to read and parse the basic clinical trial information.

In [2]:
def clinical_trial_xml_reader(file):
    """Uses BeautifulSoup to open and parse an xml file from a clinical trial.
    Returns the html/xml text.  
    
    The path and file name together are the ony argument.
    The xml_soup is returned
    """
    xml_soup = bs.BeautifulSoup(open(file,"r"), "html.parser")
    # This should automatically close the file.
    return xml_soup


def get_tag_text(soup, tag="title"):
    """A function that returns the text of the first specified tag if present,
    otherwise returns nan.

    Takes a soup of choice and the tag of choice as arguments.  Remember to put the tag in quotes.
    Returns either the text from the tag or, if the tag isn't present, NaN.
    """
    try:
        return soup.find(tag).get_text()
    except AttributeError:
        return np.NaN


def parse_clinical_trial_xml(soup, trial_data_categories):
    """A function to parse multiple myeloma clinical trials from xml files.
    Scrapes multiple fields of interest to describe the study generally.  Uses the get_tag_text()
    function to find text corresponding to tags in the list trial_data_categories.
    
    Takes as arguments the html/xml text from xml_reader() function and a list
    that acts as labels for the columns of the Series'.
    Returns a Series called 'clinical_trial_row' that can be appended as a row to a DataFrame.
    """
    category_dict = {}
    for category in trial_data_categories:
        category_dict[category] = get_tag_text(soup, category.lower())
    clinical_trial_row = pd.Series(data=category_dict, dtype=None)
    return clinical_trial_row

def clinical_trial_scrape(folder_path):
    """Uses clinical_trial_xml_reader() and parse_clinical_trial_xml functions to
    scrape basic information about all clinical trials present in the folder_path.
    If a specific field contains "nan", that means the trial did not report that
    information, which could be either improper reporting or just absence of information.
    
    Takes a folder's path as an argument.  The folder should contain .xml files from
    ClinicalTrials.gov to scrape.
    Returns a DataFrame called 'clinical_trial_df' containing basic information about
    each clinical trial as a row.
    """
    # Create a list of the categories to be scraped, 
    # and use this list as column names for a DataFrame.
    trial_data_categories = ["NCT_ID",
                             "Acronym",
                             "Brief_Title",
                             "Phase",
                             "Agency",
                             "URL",
                             "Overall_Status",
                             "Start_Date",
                             "Completion_Date",
                             "Enrollment",
                             "Number_of_Arms"]
    clinical_trial_df = pd.DataFrame(columns=trial_data_categories)
    
    # Generate a list of all .xml files in the folder,
    # then iterate over the list to parse each file.
    files = sorted([file for file in os.listdir(path) if file.endswith(".xml")])
    for file in files:
        soup = clinical_trial_xml_reader(os.path.join(path, file))
        clinical_trial_row = parse_clinical_trial_xml(soup, trial_data_categories)
        clinical_trial_df = clinical_trial_df.append(clinical_trial_row, ignore_index=True)
    
    clinical_trial_df.Start_Date = pd.to_datetime(clinical_trial_df.Start_Date)
    clinical_trial_df.Completion_Date = pd.to_datetime(clinical_trial_df.Completion_Date)
    clinical_trial_df.Enrollment = clinical_trial_df.Enrollment.astype('int64')
    
    return clinical_trial_df

The 311 .xml files were stored in the following path.

In [3]:
path = "/Users/blixt007/HTML/xml/MM_Trials"

# Basic parse of all trials

Next, the created functions from above were used to obtain a DataFrame of the clinical trial information.

In [4]:
clinical_trial_df = clinical_trial_scrape(path)

I chose to save the clinical trial information in an SQLite database to use later.

In [5]:
conn = sqlite3.connect("mm_neuro.db")
cur = conn.cursor()

clinical_trial_df.to_sql("mm_trials", con=conn, index=False, index_label="NCT_ID")
conn.commit()

Here is the scraped information from the first five trials.

In [6]:
clinical_trial_df.head()

,NCT_ID,Acronym,Brief_Title,Phase,Agency,URL,Overall_Status,Start_Date,Completion_Date,Enrollment,Number_of_Arms
0,NCT00002850,NaN,Antibiotic Therapy in Preventing Early Infecti...,Phase 3,Gary Morrow,https://clinicaltrials.gov/show/NCT00002850,Completed,1997-03-01,2012-01-01,212,3
1,NCT00006184,NaN,"Chemotherapy, Stem Cell Transplantation and Do...",Phase 2,National Cancer Institute (NCI),https://clinicaltrials.gov/show/NCT00006184,Completed,2001-02-08,2008-01-12,20,2
2,NCT00006244,NaN,"Melphalan, Peripheral Stem Cell Transplantatio...",Phase 2,Fred Hutchinson Cancer Research Center,https://clinicaltrials.gov/show/NCT00006244,Completed,2000-02-01,2016-04-01,36,1
3,NCT00027560,NaN,"Melphalan, Fludarabine, and Alemtuzumab Follow...",Phase 2,Memorial Sloan Kettering Cancer Center,https://clinicaltrials.gov/show/NCT00027560,Completed,2001-07-01,2009-04-01,51,1
4,NCT00040937,NaN,"S0204 Thalidomide, Chemotherapy, and Periphera...",Phase 2,Southwest Oncology Group,https://clinicaltrials.gov/show/NCT00040937,Completed,2002-06-01,2015-10-01,147,1


Here are the last five trials.

In [7]:
clinical_trial_df.tail()

,NCT_ID,Acronym,Brief_Title,Phase,Agency,URL,Overall_Status,Start_Date,Completion_Date,Enrollment,Number_of_Arms
306,NCT02481934,NK-VS-MM,Clinical Trial of Expanded and Activated Autol...,Phase 1,"Joaquín Martínez López, MD, PhD",https://clinicaltrials.gov/show/NCT02481934,Completed,2013-03-01,2016-10-01,5,1
307,NCT02566265,SHIVERING 2,Study of High-dose Influenza Vaccine Efficacy ...,Phase 2,Yale University,https://clinicaltrials.gov/show/NCT02566265,Completed,2015-09-01,2018-06-01,122,2
308,NCT02632786,PRONTO,"The PRONTO Study, a Global Phase 2b Study of N...",Phase 2,Prothena Therapeutics Ltd.,https://clinicaltrials.gov/show/NCT02632786,Completed,2016-03-01,2018-03-01,129,2
309,NCT02669615,NaN,Pharmacokinetic Study of Propylene Glycol-Free...,Phase 2,Medical College of Wisconsin,https://clinicaltrials.gov/show/NCT02669615,Completed,2016-11-01,2017-07-19,24,1
310,NCT03000452,FUSION-MM-005,A Study to Determine the Efficacy of the Combi...,Phase 2,Celgene,https://clinicaltrials.gov/show/NCT03000452,Completed,2017-03-14,2017-12-04,18,1


# Parsing adverse events

These following two functions were used to parse adverse events from clinical trials to help identify trials in which rates of neuropathy adverse events were high.

In [8]:
def min_max_adverse_event(path, event):
    """Determine the maximum and minimum percentage of participants in any treatment
    arm that experience the specified adverse event.
    
    Takes a path and the event as a string as agruments.
    Returns a Series of float values with the trial's NCT ID as the index.
    If the study does not report the specified adverse event, np.NaN will be returned.  
    
    Note: many studies report similar adverse events with slightly different names.
    For this reason it is best to search for the essential portion of the adverse event's 
    name instead of a very specific format.  For instance, some studies report only
    "neuropathy," while others report "neuropathy peripheral" or even "peripheral neuropathy."
    """
    
    max_adverse_event_dict = {}
    files = sorted([file for file in os.listdir(path) if file.endswith(".xml")])
    for file in files:
        soup = clinical_trial_xml_reader(os.path.join(path, file))

        adverse_events = [sub_title for sub_title in soup.find_all("sub_title") if 
                          event.lower() in sub_title.get_text().lower()]

        # Iterate over each adverse event type to find all <counts> and determine the percentage 
        # of each group with said event.
        all_adverse_event_dict = {}
        for adverse_event in adverse_events:
            counts = adverse_event.parent.find_all("counts")
            for count in counts:
                try:
                    all_adverse_event_dict[
                        (count.get("group_id") + "_" + adverse_event.get_text())] = (
                        round(int(count["subjects_affected"]
                                 )/int(count["subjects_at_risk"])*100, 2))
                
                except ZeroDivisionError:
                    all_adverse_event_dict[
                        (count.get("group_id") + "_" + adverse_event.get_text())] = np.nan

        try:
            max_adverse_event_dict[soup.nct_id.get_text()] = max(all_adverse_event_dict.values())
        except ValueError:
            max_adverse_event_dict[soup.nct_id.get_text()] = np.nan


    return pd.DataFrame([max_adverse_event_dict], index=[
            "Max % " + event.title()]).transpose()


def percent_adverse_events(path, event_list=["neuropathy", "paraesthesia"]):
    """Use the min_max_adverse_event function to parse clinical trials for 
    multiple adverse events supplied as a list.
    Returns a DataFrame with the reported minimum and maximum percentage of 
    participants who experienced each specified adverse event.
    """
    adverse_events_dataFrame = pd.DataFrame()
    for event in event_list:
        percent_event = min_max_adverse_event(path, event)
        adverse_events_dataFrame = pd.concat([adverse_events_dataFrame, percent_event], sort=False)
    return adverse_events_dataFrame

I used the same set of trial .xml files from above and parsed each one for adverse events involving any kind of neuropathy and paraesthesia.

In [9]:
path = "/Users/blixt007/HTML/xml/MM_Trials"
event_list = ["neuropathy", "paraesthesia"]
adverse_events_df = percent_adverse_events(path, event_list)

Sort the DataFrame by descending percentage of the 'Max % Neuropathy' column and display the first five trials.

In [10]:
adverse_events_df.sort_values(by="Max % Neuropathy", inplace=True, ascending=False)
adverse_events_df.head()

,Max % Neuropathy,Max % Paraesthesia
NCT00903968,100.0,NaN
NCT01246063,100.0,NaN
NCT01706666,100.0,NaN
NCT01344876,100.0,NaN
NCT01794039,100.0,NaN


# Filtering trials with lower neuropathy reports

Next I wanted to select the trials that have higher levels of any reported neuropathy event and exclude the remaining trials.

To do this, I selected the NCT_ID for every trial in which the maximum percentage of participants who experienced some form of neuropathy or paraesthesia was greater than 30 percent.

In [11]:
trial_NCT_IDs = list(adverse_events_df.loc[adverse_events_df["Max % Neuropathy"] > 30].index) + list(
        adverse_events_df.loc[adverse_events_df["Max % Paraesthesia"] > 30].index)

len(trial_NCT_IDs)

81

There were 81 out of the original 331 clinical trials in which 30 percent or more of the participants in at least one treatment arm experienced a form of neuropathy or paraesthesia.


# Detailed neuropathy-related parsing

Next I examined which adverse events in the above trials were reported in each treatment arm, and what treatments were used in each arm.

In [12]:
def parse_nervous_system_events(path, NCT_IDs):
    """Parse all reported adverse events related to the nervous system for each
    treatment arm and report the percentage of participants who experienced said
    events for each trial.
    
    Treatment arms are referred to as simple numbers, not "E#" as in the actual trial data.

    Takes the path containing the .xml files and a list of the NCT_ID numbers
    for each trial to be examined.
    Returns a DataFrame in which adverse events are columns and each treatment
    arm is an index value grouped by the NCT_ID number. Values are represented
    as percent of affected out of total per treatment arm.
    
    Note: Adverse event names can vary slightly from trial to trial, and some
    trials report many more types of adverse events than others.  Not every
    trial will have reported values for each column (adverse event) present
    in the DataFrame.  In this case, nan is reported.
    """
    adverse_event_collections = pd.DataFrame()
    for file in NCT_IDs:
        file = file + ".xml"
        soup = clinical_trial_xml_reader(os.path.join(path, file))

        # Obtain all non-serious nervous system disorders reported.
        nervous_system_disorders = [event.find_next() for event in soup.find_all(
                "title") if "Nervous" in event.get_text()]

        # If two separate fields for nervous system disorders are present, 
        # the second is non-serious events, which is what I am investigating.  
        # So drop the first instance if there are two.  If there are no fields
        # present, then no nervous system adverse events were reported, and 
        # the current trial should be skipped.
        if len(nervous_system_disorders) > 1:
            del nervous_system_disorders[0]
        elif len(nervous_system_disorders) == 0:
            continue

        # Create a nested dictionary that contains the treatment arm group and
        # percentage of participants who reported an adverse event per treatment
        # arm for each adverse event.  Use this dictionary to create a DataFrame.
        adverse_event_dict = {}
        for event in nervous_system_disorders[0].find_all("event"):
            counts_dict = {}
            for count in event.find_all("counts"):
                counts_dict[soup.find("nct_id").get_text(), count.get("group_id")[1:]] = (
                    round((int(count.get("subjects_affected"))/
                           int(count.get("subjects_at_risk"))*100), 2))
            
            # Rename neuropathy sub_titles to reduce redundancy.
            word = event.find("sub_title").get_text().lower()
            if "neuropathy" in word:
                if "peripheral" in word:
                    if "sensory" in word:
                        word = "peripheral sensory neuropathy"
                    else:
                        word = "peripheral neuropathy"
                elif "sensory" in word:
                    word = "sensory neuropathy"
                elif "motor" in word:
                    word = "motor neuropathy"
            elif "neuropath" in word:
                if "pain" in word:
                    word = "neuropathic pain"
            elif "neuro" in word:
                if "other" in word:
                    word = "other neuro"
                elif "cranial" in word:
                    word = "other neuro"
            # Remove spaces from sub_titles
            if " " in word:
                word = word.replace(" ", "_")
            
            adverse_event_dict[word.title()] = counts_dict

        adverse_event_df = pd.DataFrame(adverse_event_dict)
        adverse_event_collections = adverse_event_collections.append(adverse_event_df, sort=False)

    # Neuropathies are the main focus. Columns of adverse_event_collections are reordered
    # so all columns corresponding to neuropathies are grouped together and others are removed.
    cols = list(adverse_event_collections.columns)
    neuro_cols = []
    for col in cols:
        if "neuro" in col.lower():
            neuro_cols.append(col)
        elif "paraesthesia" in col.lower():
            neuro_cols.append(col)

    neuro_cols.sort()
    adverse_event_collections = adverse_event_collections[neuro_cols]

    return adverse_event_collections

Then I used the shortened list of clinical trials to obtain all neuro-related adverse events for these trials.  The data was stored in the same SQLite database. I also verified that my two tables are present in the SQLite database.

In [13]:
adverse_event_collections = parse_nervous_system_events(path, trial_NCT_IDs)

adverse_event_collections.to_sql(name="ae_coll", con=conn, index_label=["nct_id", "arm"])
conn.commit()
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

[('mm_trials',), ('ae_coll',)]

Summarizing the DataFrame and sorting by the highest count of each adverse event, it is clear that peripheral neuropathy and similar adverse events were reported much more frequently than others.

In [14]:
adverse_event_collections.describe().sort_values(by="count", axis=1, ascending=False)

,Peripheral_Sensory_Neuropathy,Peripheral_Neuropathy,Paraesthesia,Neuropathic_Pain,Neuropathy,Sensory_Neuropathy,Motor_Neuropathy,Other_Neuro,Polyneuropathy,Neuropathy_Cn_Iv_Down/In_Eye_Move,Neuropathy_Cn_Xii_Tongue,Neurological_Disorder_Nos,Grade_1_Cranial_Neuropathy
count,212.000000,206.000000,190.000000,66.000000,48.000000,39.000000,30.000000,20.000000,20.000000,9.000000,9.000000,8.000000,1.00
mean,20.304198,15.932718,10.545211,4.907424,4.217292,39.108462,9.977333,3.170000,3.674500,0.336667,0.336667,2.490000,7.23
std,26.797976,19.692487,15.572528,8.238332,12.400212,25.186580,10.326136,4.359469,7.776369,1.010000,1.010000,2.161666,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.23
25%,0.000000,0.000000,0.000000,0.000000,0.000000,17.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.630000,7.23
50%,6.410000,5.690000,2.315000,0.000000,0.000000,38.940000,7.750000,0.000000,0.000000,0.000000,0.000000,2.985000,7.23
75%,33.330000,30.000000,16.670000,9.452500,0.000000,57.860000,14.797500,5.387500,2.625000,0.000000,0.000000,3.330000,7.23
max,100.000000,83.330000,100.000000,25.000000,68.180000,100.000000,33.330000,15.150000,25.000000,3.030000,3.030000,6.450000,7.23


# Parsing Treatments for Each Trial

Additionally, I wanted to determine whether there were any common drugs used in studies in which the rates of neuropathy were above 30 percent.  Using the get_treatments() function lists each treatment for all clinical trials examined.

In [15]:
def get_treatments(path, NCT_IDs):
    """Creates a DataFrame relating the treatment arm number to the treatment.
    Takes a path containing clinical trial .xml files as the only argument.
    Returns a DataFrame with NCT_ID numbers as the index and treatment
    arm number as columns.
    
    Note: If a treatment arm does not exist or is not reported, the description will be replaced
    by nan.
    """
    trial_dict = {}
    for file in NCT_IDs:
        file = file + ".xml"
        soup = clinical_trial_xml_reader(os.path.join(path, file))

        # Classification of groupings for treatment arms and adverse effects
        # are nested under <reported_events>.
        group_id_dict = {}
        reported_events = soup.reported_events.find_all("group")
        n = 0
        for n in range(n, len(reported_events)):
            group_id_dict[
                int(reported_events[n].get("group_id")[1:])] = (
                reported_events[n].title.get_text())
            n+=1

        trial_dict[file[:-4]] = group_id_dict


    return pd.DataFrame(trial_dict).transpose()

In [16]:
treatment_arms = get_treatments(path, trial_NCT_IDs).sort_index()

treatment_arms.head()

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
NCT00006184,Recipient - Chemotherapy Group,Donor - Vaccination Generation Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCT00040937,Induction/PBSC Mobilization,Autologous PBSCT,Prednisone + Thalidomide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCT00064038,Lenalidomide and Dexamethasone,Dexamethasone,Crossover to Rev+Dex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCT00064337,PBSCC or Induction/PBSCC,Autologous Transplants,Dex/Thal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCT00075881,PS-341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Though the arm treatments were not well organized for an SQL table, they I still stored them in the SQLite database. To verify that the data was stored correctly, I created a new DataFrame by reading the arm treatments from the database.

In [17]:
treatment_arms.to_sql(name="treat_arms", con=conn, index_label="nct_id")
conn.commit()

treat_arms = pd.read_sql_query("SELECT * FROM treat_arms", con=conn, index_col="nct_id")

# Querying for Trial Number

Ideally, larger trials with more participants should provide better data to understand why neuropathies occur.  Using an SQLite query, I selected a subset of nct id values and treatment arms of trials that have higher than 25 percent reported neuropathic adverse events and also have a higher number of participants enrolled.  

In [18]:
cur.execute(
    """SELECT nct_id, arm FROM ae_coll
    WHERE Peripheral_Neuropathy OR
    Peripheral_Sensory_Neuropathy OR
    Sensory_Neuropathy > 25 AND
    nct_id IN (
    SELECT NCT_ID from mm_trials WHERE Enrollment > 100)
    """)
nct_id_arm = cur.fetchall()

Using the SQLite query results, I created a Series using the query as the index and the corresponding treatment from treat_arms as the values.  This was saved as a .csv file to investigate each study more closely outside of Python.  Part of the reason for this is because there was too much variation in the reported treatment arms. So graphing or analyzing the types of treatments cannot be done without extensive manipulation of reported treatments, as can be seen by the printed dictionary values below.  

In [19]:
trial_query = {pair:treat_arms.loc[pair[0], pair[1]] for pair in nct_id_arm}

pd.Series(data=trial_query).to_csv(os.path.join(path, "trial_to_investigate.csv"))

# The full list of treatments is printed below.
for value in trial_query.values():
    print(value)

Phase I - Part 1 Dose Level 0 (Carfilzomib 20/27 mg/m^2)
Phase I - Part 1 Dose Level 1 (Carfilzomib 20/36 mg/m^2)
Phase I -Part 2 Cohort 0 (Carfilzomib 56 mg/m^2+Dexamethasone)
Phase 2 (Carfilzomib 56 mg/m^2+ Dexamethasone)
Arm A
Arm C
OPB-51602 2mg/Day
OPB-51602 3mg/Day
OPB-51602 4mg/Day
OPB-51602 6mg/Day
Arm A (Lenalidomide, Dexamethasone)
Arm B (Pomalidomide, Dexamethasone)
Relapsed Myeloma (<4 Prior Regimens): Low Dose
Lenalidomide Refractory Myeloma: Low Dose
Bortezomib/Lenalidomide Refractory/Relapsed Myeloma: Low Dose
Bortezomib/Lenalidomide Relapsed/Refractory Myeloma: High Dose
Relapsed Myeloma (< 4 Prior Regimens): High Dose
Relapsed/Refractory Myeloma: High Dose
Relapsed Amyloidosis: Low Dose
Phase I - JNJ-26866138 0.7 mg/m2 Group
Phase I - JNJ-26866138 1.0 mg/m2 Group
Phase I - JNJ-26866138 1.3 mg/m2 Group
Phase II - JNJ-26866138 1.3 mg/m2 Group
Bortezomib and Thalidomide
Arm I: Thal/ZLD
Arm II: ZLD
Induction/PBSC Mobilization
Prednisone + Thalidomide
Part 1: VMP (Velcade+M

Lastly, I created a list of the .xml files from trial_query, copied each .xml file, and moved them to a new folder called "to_investigate" for further work.  

In [20]:
trials_to_investigate = set([pair[0] for pair in nct_id_arm])

os.mkdir(os.path.join(path, "to_investigate"))
for file in trials_to_investigate:
    file = file + ".xml"
    full_path = os.path.join(path, file)
    destination = os.path.join(path, "to_investigate", file)
    copyfile(full_path, destination)